# Example of support languages in Spark OCR

## Install spark-ocr python packge
Need specify path to `spark-ocr-assembly-[version].jar` or `secret`

In [1]:
secret = ""
license = ""
version = secret.split("-")[0]
spark_ocr_jar_path = "../../target/scala-2.11"

In [2]:
%%bash
if python -c 'import google.colab' &> /dev/null; then
    echo "Run on Google Colab!"
    echo "Install Open JDK"
    apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
    java -version
fi

In [3]:
import sys
import os

if 'google.colab' in sys.modules:
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

In [ ]:
# install from PYPI using secret
%pip install spark-ocr==$version\.spark24 --extra-index-url=https://pypi.johnsnowlabs.com/$secret --upgrade

In [ ]:
# or install from local path
# %pip install ../../python/dist/spark-ocr-1.11.0.spark24.tar.gz

## Initialization of spark session

In [4]:
from pyspark.sql import SparkSession
from sparkocr import start

if license:
    os.environ['JSL_OCR_LICENSE'] = license

spark = start(secret=secret, nlp_version="2.7.0", jar_path=spark_ocr_jar_path)
spark

Spark version: 2.4.4
Spark NLP version: 2.6.5
Spark OCR version: 1.11.0



## Import OCR transformers

In [5]:
from sparkocr.transformers import *
from sparkocr.enums import *
from pyspark.ml import PipelineModel

## Awailable languages

In [61]:
for lang in Language:
    print(lang)

Language.ENG
Language.FRA
Language.RUS
Language.DEU
Language.SPA


## Define OCR transformers and pipeline

In [62]:
def pipeline():
    
    binary_to_image = BinaryToImage()
    binary_to_image.setInputCol("content")
    binary_to_image.setOutputCol("image")
    
    # Run OCR
    ocr = ImageToText()
    ocr.setInputCol("image")
    ocr.setOutputCol("text")
    # Set russian language
    ocr.setLanguage(Language.RUS)
    # Set model type: BASE, BEST, FAST
    ocr.setModelType(ModelType.BEST)
    # Download model from JSL S3
    ocr.setDownloadModelData(True)
    
    pipeline = PipelineModel(stages=[
        binary_to_image,
        ocr
    ])
    
    return pipeline

## Read image as binary file

In [20]:
import pkg_resources
image_example = pkg_resources.resource_filename('sparkocr', 'resources/ocr/images/langs/rus.png')
image_example_df = spark.read.format("binaryFile").load(image_example).cache()

## Run OCR pipelines

In [63]:
result = pipeline().transform(image_example_df).cache()

## Display results

In [64]:
result.select("pagenum","text", "confidence").show()

+-------+--------------------+-----------------+
|pagenum|                text|       confidence|
+-------+--------------------+-----------------+
|      0|Определить свою ц...|94.28230394635882|
+-------+--------------------+-----------------+



### Display recognized text

In [65]:
print("\n".join([row.text for row in result.select("text").collect()]))

Определить свою цель — это как найти Полярную звезду.
Суть жизни — найти самого себя.

Не стоит принимать доброту за слабость, грубость за силу, а подлость
за умение жить.

Если проигравший улыбается, победитель теряет вкус победы.
На самом деле, жизнь проста, но мы настойчиво её усложняем.

Кон ий

Если вы хотите иметь то, что никогда не имели, вам придётся делать
то, что никогда не делали.

Никто не ценит того, чего слишком много.

Каждая мелочь может послужить твоему вдохновению.



## Clear cache

In [ ]:
result.unpersist()